## MCP (Model Context Protocol) 教程

在这个notebook中，我们将学习如何使用MCP (Model Context Protocol) 来创建可复用的工具服务器，并让AI Agent通过标准化协议调用这些工具。

MCP是一个开放标准，允许AI应用程序与外部数据源和工具进行安全连接。它提供了一种统一的方式来暴露工具、资源和提示给AI模型。

### 核心概念

1. **MCP服务器**: 暴露工具和资源的服务
2. **MCP客户端**: 连接到MCP服务器并使用其功能的应用程序
3. **传输层**: 服务器和客户端之间的通信方式（stdio、HTTP等）
4. **工具**: 可以被AI调用的函数
5. **资源**: 可以被AI访问的数据源


## MCP 服务器
调用者和agent不关心实现的细节，只关心暴露出什么工具

In [ ]:
# weather_server.py
from mcp.server.fastmcp import FastMCP

# 创建MCP服务器实例
mcp = FastMCP("Weather")

@mcp.tool()
async def get_weather(location: str) -> str:
    """获取指定地点的天气信息
    
    Args:
        location: 地点名称
    
    Returns:
        天气信息字符串
    """
    # 这里是模拟的天气数据
    weather_data = {
        "New York": "晴朗，温度25°C",
        "London": "多云，温度18°C", 
        "Tokyo": "雨天，温度22°C",
        "Beijing": "晴朗，温度28°C"
    }
    
    return weather_data.get(location, f"无法获取{location}的天气信息")

@mcp.tool()
async def get_forecast(location: str, days: int = 3) -> str:
    """获取指定地点的天气预报
    
    Args:
        location: 地点名称
        days: 预报天数，默认3天
    
    Returns:
        天气预报信息
    """
    forecast = []
    for i in range(days):
        forecast.append(f"第{i+1}天: {location} - 晴朗，温度{20+i*2}°C")
    
    return "\n".join(forecast)

if __name__ == "__main__":
    # 使用streamable-http传输方式启动服务器
    mcp.run(transport="streamable-http")
    


In [ ]:
# math_server.py
from mcp.server.fastmcp import FastMCP

# 创建数学运算服务器
mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    """加法运算
    
    Args:
        a: 第一个数
        b: 第二个数
    
    Returns:
        两数相加的结果
    """
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """乘法运算
    
    Args:
        a: 第一个数
        b: 第二个数
    
    Returns:
        两数相乘的结果
    """
    return a * b

@mcp.tool()
def subtract(a: int, b: int) -> int:
    """减法运算
    
    Args:
        a: 被减数
        b: 减数
    
    Returns:
        减法结果
    """
    return a - b

@mcp.tool()
def divide(a: float, b: float) -> float:
    """除法运算
    
    Args:
        a: 被除数
        b: 除数
    
    Returns:
        除法结果
    """
    if b == 0:
        raise ValueError("除数不能为0")
    return a / b

if __name__ == "__main__":
    # 使用stdio传输方式启动服务器
    mcp.run(transport="stdio")


## 使用LangChain来调用MCP服务

In [ ]:
# mcp_client.py
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

async def setup_mcp_client():
    """设置MCP客户端，连接到多个服务器"""
    
    # 配置多个MCP服务器
    client = MultiServerMCPClient(
        {
            "math": {
                "command": "python",
                # 替换为你的math_server.py文件的绝对路径
                "args": ["/path/to/math_server.py"],
                "transport": "stdio",
            },
            "weather": {
                # 确保天气服务器在8000端口运行
                "url": "http://localhost:8000/mcp",
                "transport": "streamable_http",
            }
        }
    )
    
    # 获取所有可用的工具
    tools = await client.get_tools()
    
    print("可用的工具:")
    for tool in tools:
        print(f"- {tool.name}: {tool.description}")
    
    return client, tools

async def create_agent_with_mcp_tools():
    """创建带有MCP工具的Agent"""
    
    client, tools = await setup_mcp_client()
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    # 创建ReAct Agent
    agent = create_react_agent(
        llm,  # 或者其他支持的模型
        tools
    )
    
    return agent

async def test_mcp_agent():
    """测试MCP Agent的功能"""
    
    agent = await create_agent_with_mcp_tools()
    
    # 测试数学运算
    math_response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "计算 (3 + 5) x 12 是多少？"}]}
    )
    print("数学运算结果:")
    print(math_response["messages"][-1]["content"])
    
    # 测试天气查询
    weather_response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "纽约的天气怎么样？"}]}
    )
    print("天气查询结果:")
    print(weather_response["messages"][-1]["content"])
    
    # 测试复合查询
    complex_response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "如果北京今天的温度是28度，东京的温度是22度，它们的温度差是多少？"}]}
    )
    print("复合查询结果:")
    print(complex_response["messages"][-1]["content"])

# 运行示例
if __name__ == "__main__":
    asyncio.run(test_mcp_agent())


### 传输方式详解

MCP支持两种主要的传输方式：

#### 1. Standard I/O (stdio)
- **使用场景**: 适用于本地进程间通信
- **启动方式**: `mcp.run(transport="stdio")`
- **优点**: 简单、快速、安全
- **缺点**: 只能在本地使用

#### 2. Streamable HTTP
- **使用场景**: 适用于网络通信，可以远程访问，例如我们的NLE MCP服务
- **启动方式**: `mcp.run(transport="streamable-http")`
- **优点**: 支持远程访问、可以通过网络调用
- **缺点**: 需要处理网络安全问题

### 运行服务器的方式

```bash
# 启动天气服务器（HTTP方式）
python weather_server.py

# 启动数学服务器（stdio方式）
python math_server.py
```

### MCP与传统工具调用的比较

#### 传统工具调用方式
```python
# 传统方式：直接定义工具函数
@tool
def get_weather(location: str) -> str:
    return "It's sunny"

tools = [get_weather]
agent = create_react_agent(model, tools)
```

#### MCP方式
```python
# MCP方式：通过协议调用远程工具
client = MultiServerMCPClient(servers_config)
tools = await client.get_tools()
agent = create_react_agent(model, tools)
```

### MCP的优势

#### 1. 模块化和可复用性
- **传统方式**: 工具函数与应用程序耦合
- **MCP方式**: 工具可以独立开发、部署和维护

#### 2. 分布式架构
- **传统方式**: 所有工具必须在同一个进程中
- **MCP方式**: 工具可以分布在不同的服务器上

#### 3. 标准化接口
- **传统方式**: 每个应用都有自己的工具定义方式
- **MCP方式**: 统一的协议标准，工具可以在不同应用间共享

#### 4. 安全性
- **传统方式**: 工具直接在应用程序中执行
- **MCP方式**: 可以通过网络隔离和权限控制提高安全性

#### 5. 扩展性
- **传统方式**: 添加新工具需要修改应用程序
- **MCP方式**: 可以动态发现和使用新的工具服务器


### 总结

MCP为AI应用提供了一种标准化、模块化的工具集成方式。它特别适合构建复杂的、分布式的AI系统，不同的工具可以独立开发、部署和维护。

通过MCP，我们可以：
- 创建可复用的工具服务器
- 构建分布式AI架构
- 实现工具的标准化接口
- 提高系统的可维护性和扩展性

但是，对于本地可以执行且没有复用需求的工具，我们还是倾向于直接在本地定义并直接在agent中调用。